In [1]:
import requests
from pyquery import PyQuery as pq
from requests_toolbelt import MultipartEncoder

In [2]:
baseurl = "https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf"

In [3]:
r = requests.get(baseurl)
html = str(r.content)
cookies = r.cookies

In [4]:
print(html)

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en" dir="ltr"><head id="j_idt2">\n\t\t\t<meta http-equiv="X-UA-Compatible" content="IE=9" /><link type="text/css" rel="stylesheet" href="/ocr/javax.faces.resource/theme.css.jsf?ln=primefaces-aristo" /><link type="text/css" rel="stylesheet" href="/ocr/javax.faces.resource/components.css.jsf;jsessionid=8E513D38207FAE1E70BBA7859B7A2F75?ln=primefaces&amp;v=6.1" /><script type="text/javascript" src="/ocr/javax.faces.resource/jquery/jquery.js.jsf;jsessionid=8E513D38207FAE1E70BBA7859B7A2F75?ln=primefaces&amp;v=6.1"></script><script type="text/javascript" src="/ocr/javax.faces.resource/jquery/jquery-plugins.js.jsf;jsessionid=8E513D38207FAE1E70BBA7859B7A2F75?ln=primefaces&amp;v=6.1"></script><script type="text/javascript" src="/ocr/javax.faces.resource/core.js.jsf;jsessionid=8E513D38207FAE1E70BBA7859B7A2F75?ln=primefaces&amp;v=6.1"></script><script type="text/javascript" src="/ocr/javax.faces.resource/components.j

In [5]:
pq(html)("input")

[<input>, <input#j_id1:javax.faces.ViewState:0>, <input>, <input#j_id1:javax.faces.ViewState:1>]

In [6]:
inputs = pq(html)("input")
pagedict = {}
for input in inputs:
    # print(pq(input))
    if pq(input).attr("type") == "hidden":
        name = pq(input).attr("name")
        value = pq(input).attr("value")
        pagedict[name] = value



In [ ]:
# One thing to note: That last ID you set (`ocrForm:j_idt364`) 
# has changed a few times over the years. Don't know why, 
# but it might be worth scraping for that string as well. 
# (Basically, I look for the "Export as XML" button 
# (or whichever you want, but that's what I'm using) 
# and pull the ID out of that button `onclick` handler Tim showed earlier.)

In [7]:
pagedict["ocrForm:reportResultTable_rppDD"] = '100'
pagedict["ocrForm:j_idt364"] = "ocrForm:j_idt364"

In [8]:
print(pagedict)

{'headerNavigation': 'headerNavigation', 'javax.faces.ViewState': '4303041469993166563:5042114302048694886', 'ocrForm': 'ocrForm', 'ocrForm:reportResultTable_rppDD': '100', 'ocrForm:j_idt364': 'ocrForm:j_idt364'}


In [9]:
payload = {}
del pagedict['headerNavigation']
for item in pagedict:
    payload[item] = (None, pagedict[item])
m = MultipartEncoder(fields=payload)
r = requests.post(baseurl, data=m, headers={'Content-Type': m.content_type}, cookies=cookies)

In [10]:
m

<MultipartEncoder: {'javax.faces.ViewState': (None, '4303041469993166563:5042114302048694886'), 'ocrForm': (None, 'ocrForm'), 'ocrForm:reportResultTable_rppDD': (None, '100'), 'ocrForm:j_idt364': (None, 'ocrForm:j_idt364')}>

In [11]:
print(payload)

{'javax.faces.ViewState': (None, '4303041469993166563:5042114302048694886'), 'ocrForm': (None, 'ocrForm'), 'ocrForm:reportResultTable_rppDD': (None, '100'), 'ocrForm:j_idt364': (None, 'ocrForm:j_idt364')}


In [12]:
with open('test.csv', 'wb') as f:
    f.write(r.content)